In [1]:
import sys
import os

# 将当前notebook所在目录（即本文件的上层目录）加入到sys.path中
notebook_dir = "/Users/muwang/Documents/github/TomoPANDA-pick/"
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)






In [2]:
from utils.io_dynamo import dynamo_tbl_vll_to_relion_star

In [8]:
project_dir = '/Users/muwang/Documents/github/TomoPANDA-pick/'
vll_path = os.path.join(project_dir, 'notebooks/input/dcp_single_030/crop.vll')
tbl_path = os.path.join(project_dir, 'notebooks/input/dcp_single_030/refined_table_ref_001_ite_0004.tbl')
rl_particles_path = os.path.join(project_dir, 'notebooks/dcp_single_030/refined_table_ref_001_ite_0004.star')

In [4]:
dynamo_tbl_vll_to_relion_star(tbl_path, vll_path, output_file=rl_particles_path,bin_scalar=8.0,pixel_size=0.84,tomogram_size=(999,999,499),output_centered=True)

Using rlnMicrographName column
Converted 21038 particles from Dynamo format to RELION format.
Output written to: /Users/muwang/Documents/github/TomoPANDA-pick/notebooks/dcp_single_030/refined_table_ref_001_ite_0004.star


,rlnCenteredCoordinateXAngst,rlnCenteredCoordinateYAngst,rlnCenteredCoordinateZAngst,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnOriginXAngst,rlnOriginYAngst,rlnOriginZAngst,rlnMicrographName
0,1337.70,-46.62,1722.42,-133.444,70.291,102.0480,0.0,0.0,0.0,rec_MGS5_001_T1_ts_001
1,5772.90,1458.66,378.42,-8.762,115.980,-73.6700,0.0,0.0,0.0,rec_MGS5_001_T1_ts_001
2,1391.46,-6.30,1561.14,-133.398,63.649,107.8360,0.0,0.0,0.0,rec_MGS5_001_T1_ts_001
3,5746.02,611.94,203.70,-11.767,127.390,74.6970,0.0,0.0,0.0,rec_MGS5_001_T1_ts_001
4,5786.34,1808.10,459.06,-23.314,121.530,-113.3500,0.0,0.0,0.0,rec_MGS5_001_T1_ts_001
...,...,...,...,...,...,...,...,...,...,...
21033,3125.22,490.98,620.34,117.010,114.590,-138.8000,0.0,0.0,0.0,rec_pinkss_MGS002_T2_ts_010
21034,1955.94,181.86,2354.10,3.008,89.142,-99.8200,0.0,0.0,0.0,rec_pinkss_MGS002_T2_ts_010
21035,3528.42,1418.34,297.78,-75.336,80.084,-96.0400,0.0,0.0,0.0,rec_pinkss_MGS002_T2_ts_010
21036,5342.82,302.82,633.78,-47.718,89.167,82.5553,0.0,0.0,0.0,rec_pinkss_MGS002_T2_ts_010


In [9]:
# read the star file by starfile
import starfile
rl_particles = starfile.read(rl_particles_path)
# add a new column 'rlnTomoName' to the star file, set the value from 1 based on the rlnMicrographName column. 不同的MicrographName顺次编号
# 利用每个不同的 rlnMicrographName 生成不同的序号，从1开始
import pandas as pd

# 获取唯一的MicrographName，并建立映射关系
unique_micrographs = rl_particles['rlnMicrographName'].unique()
micrograph_to_tomo_number = {name: idx+1 for idx, name in enumerate(unique_micrographs)}

# 根据映射为每行赋予编号
rl_particles['rlnTomoName'] = rl_particles['rlnMicrographName'].map(micrograph_to_tomo_number)

# write the star file
rl_particles_path = rl_particles_path.replace('.star', '_tomo_name.star')
starfile.write(rl_particles, rl_particles_path)
